In [ ]:
# install dependencies
# use pip and virtualenv
!python --version
!pip install nltk matplotlib prettytable tqdm
!pip install langchain_community langchain-huggingface langchain-chroma chromadb==0.5.5 sentence_transformers pypdf

In [ ]:
# global vars
TRAINING_DATA_PATH = "training_data.local/"

# import libraries
try:
    from langchain_core.documents import Document
    from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
    from langchain_text_splitters import RecursiveCharacterTextSplitter as rts
    from langchain_text_splitters import NLTKTextSplitter as nts
    from prettytable import PrettyTable
    from langchain_chroma import Chroma
    from chromadb import Client
    import uuid
    from tqdm.autonotebook import tqdm, trange
    from nltk import word_tokenize, sent_tokenize, download
    import os
    download("punkt")
    import matplotlib.pyplot as plt
except Exception as e:
    print(f"Caught Exception {e}")

In [ ]:
# Text Processing Primitives
# Embedding function
def s_transformer(model: str = "all-MiniLM-L6-v2", hf=False):
  if hf is True:
    from langchain_huggingface import HuggingFaceEmbeddings as hfe
    return hfe(model_name=model)
  else:
    from chromadb.utils import embedding_functions as ef
    return ef.SentenceTransformerEmbeddingFunction(model_name=model)

# instantiate loader
def new_loader(path: str = ".", pattern: str = "**/*.txt",
               loader_class=None, l_kwargs: dict = None, multithread: bool = False) -> DirectoryLoader:

    if loader_class is None:
      loader_class = TextLoader

    return DirectoryLoader(path, glob=pattern,
                           loader_cls=loader_class,
                           loader_kwargs=l_kwargs,
                           use_multithreading=multithread,
                           silent_errors=True,
                           show_progress=True)

# load text documents from filesystem
def load_text_documents(path: str = ".", pattern: str = "**/*.txt",
                        multithread: bool = False) -> list:
    loader = new_loader(path=path,
                        pattern=pattern,
                        loader_class=TextLoader,
                        l_kwargs={'autodetect_encoding': True},
                        multithread=multithread)
    return loader.load()

# load PDF documents from filesystem
def load_pdf_documents(path: str = ".", pattern: str = "**/*.pdf",
                        multithread: bool = False) -> list:
    loader = new_loader(path=path,
                        pattern=pattern,
                        loader_class=PyPDFLoader,
                        multithread=multithread)
    return loader.load()

# prepare the knowledge corpus for further processing
def prepare_corpus(raw_loader: list) -> list:
    # collect statistics in relation to the data corpus
    metadata = []
    for doc in raw_loader:
        document_tokens = word_tokenize(doc.page_content)
        document_sentences = sent_tokenize(doc.page_content)
        if (len(document_tokens) != 0) and (len(document_sentences) != 0):
          metadata.append({"metadata": doc.metadata,
                        "raw_sentences": document_sentences,
                        "page_contents": doc.page_content,
                        "sentence_count": len(document_sentences),
                        "wordcount": len(document_tokens),
                        "vocabulary": set(document_tokens),
                        "lexical_richness": len(set(document_tokens)) / len(document_tokens)})

    # display corpus
    data_table = PrettyTable()
    data_table.field_names = ["Dataset", "Word Count", "Sentence Count", "Vocabulary", "Lexical Richness"]
    for dataset in metadata:
        data_table.add_row([dataset.get("metadata").get("source"), dataset.get("wordcount"), dataset.get("sentence_count"), len(dataset.get("vocabulary")), dataset.get("lexical_richness")])

    # display dataset statistics
    print(data_table)

    # return processed data
    corpus_data = []
    for doc in tqdm(metadata, ascii=True, desc="Loading..."):
        doc_metadata = doc.get("metadata")
        corpus_data.append(Document(metadata=doc_metadata, page_content=doc.get("page_contents")))

    # return corpus and metadata
    return corpus_data, metadata

# split corpus in chunks for vectorization
def split_text_documents_recursive(documents: list = None,
                         chunk_size: int = 1000,
                         chunk_overlap: int = 0) -> list:
    if documents is None:
        return None

    splitter = rts(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)

# split corpus using NLTK
def split_text_documents_nltk(documents: list = None,
                              separator: str = '\n\n',
                              language: str = 'english') -> list:
    if documents is None:
      return None

    splitter = nts(separator=separator, language=language)
    return splitter.split_documents(documents)

In [ ]:
# walk the raw data storage path and search for text documents
raw_text_loader = load_text_documents(TRAINING_DATA_PATH)
raw_pdf_loader = load_pdf_documents(TRAINING_DATA_PATH)

# collect statistics in relation to the data corpus
corpus_data, metadata = prepare_corpus(raw_text_loader + raw_pdf_loader)

In [ ]:
# plot dataset statistics graph
x_vals = [len(x.get("vocabulary")) for x in metadata]
y_vals = [100 * x.get("lexical_richness") for x in metadata]
y2_vals = [x.get("sentence_count") for x in metadata]

# plot vocabulary vs richness data
plt.subplot(211)
plt.ylabel("Lexical Richness")
plt.xlabel("Vocabulary")
plt.scatter(x_vals, y_vals)
# plot sentence count vs vocabulary
plt.subplot(212)
plt.ylabel("Sentence Count")
plt.xlabel("Vocabulary")
plt.scatter(x_vals, y2_vals)
plt.show()

In [ ]:
# prepare data for further tokenization
CHUNK_SIZE = 2000 # 1000 words per chunk
OVERLAP = 200 # overlapping words, avoid missing context and splitting sentences
SEPARATOR='\n\n'
TEXT_LANGUAGE='english'

# split documents (recursive)
tokenized_docs_recursive = split_text_documents_recursive(corpus_data, chunk_size=CHUNK_SIZE, chunk_overlap=OVERLAP)

# split documents (NLTK)
tokenized_docs_nltk = split_text_documents_nltk(corpus_data, separator=SEPARATOR, language=TEXT_LANGUAGE)

print(f"Tokenized {len(tokenized_docs_recursive)} documents via Recursive Splitter...")
print(f"Tokenized {len(tokenized_docs_nltk)} documents via NLTK Splitter...")

# remove buffers
del(raw_text_loader)
del(raw_pdf_loader)
del(corpus_data)
del(metadata)

In [ ]:
# now call the embedding model and upload data to the vector database
SIMILARITY_FUNCTION = "cosine"
COLLECTION_NAME = "rag_demo"
os.environ['HF_HOME'] = '/tmp/huggingface/hub/'

# connect to a running chroma instance
try:
    vector_store_client = Client()
    chroma_collection = vector_store_client.get_or_create_collection(COLLECTION_NAME,
                                                              metadata={"hnsw:space": SIMILARITY_FUNCTION})
except Exception as e:
    print(f"Caught Exception: {e}")

In [ ]:
# create a wrapper around the ChromaDB client to use via LangChain
chroma_langchain_adapter = Chroma(client=vector_store_client, collection_name=COLLECTION_NAME, embedding_function=s_transformer(hf=True))

In [ ]:
# embed data and push vectors to the database (NLTK tokens)
if len(tokenized_docs_nltk) > 0:
  #chroma_collection.add(ids=[str(uuid.uuid1()) for doc in tokenized_docs], documents=[doc.page_content for doc in tokenized_docs], metadatas=[doc.metadata for doc in tokenized_docs])
    for doc in tqdm(tokenized_docs_nltk, ascii=True, desc="Ingesting..."):
      chroma_langchain_adapter.add_documents(ids=[str(uuid.uuid1())], documents=[doc])

In [ ]:
# define query example
QUERY_TEXT="disassembly in x86_64 asm language"

# explore the vector collection
print(f"Objects stored in the collection {chroma_collection.name}: {chroma_collection.count()}")
chroma_collection.query(query_texts=[QUERY_TEXT], n_results=1)

# query via LangChain
results = chroma_langchain_adapter.similarity_search_with_score(
  QUERY_TEXT, k=5,
)

# build a vector table
retrieved_docs = {}
for res, score in results:
  retrieved_docs[score] = res
  print(f"* {score:3f} - [{res.metadata}]")

# now get the result with the higher similarity (lower score = more similarity)
best_doc_by_similarity_score = retrieved_docs[min(retrieved_docs.keys())]
print(f"BEST DOCUMENT - Score {min(retrieved_docs.keys())} - {best_doc_by_similarity_score.metadata}")

In [ ]:
del chroma_langchain_adapter
vector_store_client.delete_collection(COLLECTION_NAME)